# Notebook to generate figure 1

In [ ]:
# imports and definitions
import Utils
import os
import numpy as np
import pandas as pd
%matplotlib inline
from pandas.plotting import scatter_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l1,l2
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.models import load_model
from ML import *

ph_dir = "/home/pau/Data/BioB/"
gn_dir = '/home/data/biobank/'
force_recreation=False
#Load Data 
x_train, x_test, y_train, y_test = Utils.load_data(ph_dir, gn_dir, trait="height", th=65.975)

In [ ]:
def Train(x_train,y_train,model,k=0,v=0):
    early_stopping = EarlyStopping(patience=4, verbose=0)
    reduce_lr = ReduceLROnPlateau(factor=0.2, verbose=0, patience=2, min_lr=1e-5)
    callbacks=[reduce_lr, early_stopping]
    model.fit(x_train, y_train, callbacks=callbacks, epochs=150, validation_split=0.2, verbose=v)
    return model

In [ ]:
# Load / Train models
if os.path.exists('models/Gwas_10k/mlp1.hd5') and not force_recreation:
    m1 = load_model('models/Gwas_10k/mlp1.hd5')
else:
    m1 = model1(x_train)
    m1 = Train(x_train,y_train,m1)
    m1.save('models/Gwas_10k/mlp1.hd5')
    
if os.path.exists('models/Gwas_10k/mlp2.hd5') and not force_recreation:
    m2 = load_model('models/Gwas_10k/mlp2.hd5')
else:
    m2 = ridge(x_train)
    m2 = Train(x_train,y_train,r)
    m2.save('models/Gwas_10k/mlp2.hd5') 

if os.path.exists('models/Gwas_10k/lasso.hd5') and not force_recreation:
    l = load_model('models/Gwas_10k/lasso.hd5')
else:
    l = lasso(x_train)
    l = Train(x_train,y_train,l)
    m1.save('models/Gwas_10k/lasso.hd5')    
    
if os.path.exists('models/Gwas_10k/ridge.hd5') and not force_recreation:
    r = load_model('models/Gwas_10k/ridge.hd5')
else:
    r = ridge(x_train)
    r = Train(x_train,y_train,r)
    m1.save('models/Gwas_10k/ridge.hd5') 

## Z-score feature importance and hidden representations

In [ ]:
def zImp(model):
    nf = model.input_shape[1]
    x = np.eye(nf)
    y = model.predict(x).ravel()
    y = (y - y.mean())/y.std()
    y = np.abs(y)
    y = y / y.sum()
    return y

In [ ]:
def getH1(model,x):
    m = Model(model.layers[0].input,model.layers[0].output)
    return m.predict(x)

In [ ]:
# compute importances of features for different methods
imp = zImp(m1)
imp2 = zImp(m2)
wl = np.abs(l.get_weights()[0])
wl = wl/wl.sum()
wr = np.abs(mr.get_weights()[0])
wr = wr/wr.sum()
m = np.asarray([wl.ravel()*100,wr.ravel()*100,imp.ravel()*100,imp2.ravel()*100])
m = m.transpose()

In [ ]:
# Produce Varible importance scatter plot
axes = scatter_matrix(df, alpha=0.5,figsize=(10, 10), diagonal='kde')
corr = df.corr().as_matrix()
for i, j in zip(*plt.np.triu_indices_from(axes, k=1)):
    axes[i, j].annotate("%.3f" %corr[i,j], (0.8, 0.8), xycoords='axes fraction', ha='center', va='center')
plt.suptitle('Varible importance of different models: GWAS 10k')
plt.savefig('imp_10k.png')

In [ ]:
# First layer representation mlp1
h1_mlp1 = getH1(m1,x_train)
idx= sum(h1_mlp1>0).argsort()[::-1]
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
z1=ax1.hist(h1_mlp1[:,idx[0]]*50,50,density=True)
ax1.set_title('1st')
ax1.set_xlabel('activation value')
z2=ax2.hist(h1_mlp1[:,idx[1]]*50,50,density=True)
ax2.set_title('2nd')
ax2.set_xlabel('activation value')
z3=ax3.hist(h1_mlp1[:,idx[2]]*50,50,density=True)
ax3.set_title('3rd')
ax3.set_xlabel('activation value')
f.suptitle("Most active neurons 1st layer MLP1")
f.savefig('plots/mlp1_1st_10k.png',transparent=True)

In [ ]:
# First layer representation mlp1
h1_mlp2=getH1(m2,x_train)
idx= sum(h1_mlp2>0).argsort()[::-1]
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
a1=ax1.hist(h1_mlp2[:,idx[0]],50,density=True)
ax1.set_title('1st')
ax1.set_xlabel('activation value')
a2=ax2.hist(h1_mlp2[:,idx[1]],50,density=True)
ax2.set_title('2nd')
ax2.set_xlabel('activation value')
a3=ax3.hist(h1_mlp2[:,idx[2]],50,density=True)
ax3.set_title('3rd')
ax3.set_xlabel('activation value')
f.suptitle("Most active neurons 1st layer mlp2")
f.savefig('plots/mlp2_1st_10k.png',transparent=True)